# FINAL PROJECT DELIVERABLE  

I have decided to do some major changes to how I am going to approach this Query Machine.

the dataset that I have created in the initial D2 contains global indexes of living indecators for alot of countries (about 150 countries), after rethinking i have taken two choices on how to deal with this dataset. These changes are:

1- decrease the number of countries majorly (to ~50 countries), this is because most countries that exist within the same region have very similar metrics, this will make the classification task very hard and cause potential overfittiing for the data, which will even produce intersecting clusters. the countries will be chosen stratigically so that the dataset contains the hightest number of people (so i will include countries with high populations), also each region will have at least two countries (choosing qatar and KSA for example for the gulf ragion). This will ensure that the data prediction will have a higher accuracy.
    
2- use the data metrics from the chosen countires initial D2 dataset as random data generating points ( I will use each metric to be the median of a normal distribution), this will ensure to simulate a real life questionnaire data set according to the metrics that these data points were initially based on.  
    
    

## 1- D2 FINAL 

In [1]:
import pandas as pd
import matplotlib
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.dpi']= 100
import numpy as np
import time
from sklearn import cluster
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from scipy.stats import mode
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import numpy as np

ModuleNotFoundError: No module named 'keras'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
dg = pd.read_csv("data-source-grand1.csv")
df = pd.read_csv("data-freedom.csv")

In [ ]:
#adding freedom to the dg dataset
for i in range(156):
    if(dg["Country or region"][i] in list(df['country'])):
        n = df[df["country"]==dg["Country or region"][i]].index.values
        dg["freedom"][i] = df['freedom index'][n]

In [ ]:
#checking for nans
np.sum(dg.isna().sum())

I have decided to drop all nan data, this is because I am trying to minimize the countries included, dropping countries with nan fields will help with that

In [ ]:
dg = dg.dropna(axis='columns')

In [ ]:
dg

for choosing the countries, I am going to keep only the top 50 countries in terms of population, this will help decrease the error and build a (somewhat) accurate model.

These countries are:
China,
India,
United States,
Indonesia,
Pakistan,
Brazil,
Nigeria,
Bangladesh,
Russia,
Mexico,
Japan,
Ethiopia,
Philippines,
Egypt,
Vietnam,
DR Congo,
Turkey,
Iran,
Germany,
Thailand,
United Kingdom,
France,
Italy,
Tanzania,
South Africa,
Myanmar,
Kenya,
South Korea,
Colombia,
Spain,
Uganda,
Argentina,
Algeria,
Sudan,
Ukraine,
Iraq,
Afghanistan,
Poland,
Canada,
Morocco,
Saudi Arabia,
Uzbekistan,
Peru,
Angola,
Malaysia,
Mozambique,
Ghana,
Yemen,
Nepal.

these countries represent 86.42% of the global world population

I will drop all countries that are not in the list of the top 50 in population

In [ ]:
#dataset with top 50 countries in terms of population (Qatar is added because why not)
top50 = pd.read_csv("top50.csv")

In [ ]:
top50

In [ ]:
top50_countries = top50["Country (or dependency)"]

In [ ]:
top50_countries = top50_countries.drop(51) #cleaning the data

In [ ]:
top50_countries

In [ ]:
#cleaning the dataset to inlude the top 50 in terms of population
#dg = dg.drop(6)
for i in range(len(dg["Country or region"])):
    if(dg["Country or region"][i] not in list(top50_countries)):
        dg = dg.drop(i)

In [ ]:
len(dg)

In [ ]:
dg

now that the dataset is finally complete, it is time to explore it and fine-tune it

from the data set, the ranking field will be dropped because it does not serve a purpose, also the data needs to be rescaled on a scale 0-10, this scale is chosen because it is common and easier for people to classify based on such scale. random data that will be normally distributed will be generated based on the indices, these will be the best in simulating an in real life questionnaire, also will be helpfull in creating a K-NN classifier for the data.

In [ ]:
dg = dg.drop("rank",axis =1)
dg = dg.drop("Score",axis = 1)

In [ ]:
dg = dg.drop(102) #i will remove nigeria because it overshadows other countries in the classifier

In [ ]:
#exploring the data
#GPD value :
print("GPD per capita ranges from",min(dg["GDP per capita"]),"to",max(dg["GDP per capita"]))
#social support:
print("social support ranges from",min(dg["Social support"]),"to",max(dg["Social support"]))
#Healthy life expectancy:
print("Healthy life expectancy ranges from",min(dg["Healthy life expectancy"]),"to",max(dg["Healthy life expectancy"]))
#Freedom to make life choices
print("Freedom to make life choices ranges from",min(dg["Freedom to make life choices"]),"to",max(dg["Freedom to make life choices"]))
#Generosity
print("Generosity ranges from",min(dg["Generosity"]),"to",max(dg["Generosity"]))
#Perceptions of corruption
print("Perceptions of corruption ranges from",min(dg["Perceptions of corruption"]),"to",max(dg["Perceptions of corruption"]))

In [ ]:
gdf = pd.DataFrame()
gdict = {"country":[],"GDP":[],"social support":[],"healthy life":[],"freedom of choice":[],"generosity":[],"corruption":[]}
for i in range(len(dg)):
    for j in range(100):
        gdict["country"] += [list(dg["Country or region"])[i]]
        gdict["GDP"] += [np.random.normal(list(dg["GDP per capita"])[i],0.1,1)]
        gdict["social support"] += [np.random.normal(list(dg["Social support"])[i],0.1,1)]
        gdict["healthy life"] += [np.random.normal(list(dg["Healthy life expectancy"])[i],0.1,1)]
        gdict["freedom of choice"] += [np.random.normal(list(dg["Freedom to make life choices"])[i],0.1,1)]
        gdict["generosity"] += [np.random.normal(list(dg["Generosity"])[i],0.1,1)]
        gdict["corruption"] += [np.random.normal(list(dg["Perceptions of corruption"])[i],0.1,1)]        

In [ ]:
gdf = pd.DataFrame(gdict)

In [ ]:
#correcting the values format
for i in range(len(gdf)):
    for j in range(6):
        gdf.loc[i][j+1] = gdf.loc[i][j+1][0] +1 #shifiting the data so the normal distribution wont give negative values

In [ ]:
gdf.describe()

In [ ]:
gdf

In [ ]:
gdf = gdf.sample(frac = 1)

In [ ]:
#creating historgrams usong seaborn
import seaborn as sb

sb.set(style="ticks", color_codes=False)

g = sb.pairplot(gdf, hue="country", diag_kind='hist')

1- GDP seems like the factor with highest correlation with all other factors

2- there seems to be a good correlation between social support and GDP, healthy life and freedom of choice.

3- freedom of choice to be sparsely distributed making it hard to create correlaion from it, it might get dropped

In [ ]:
#creating histogram using matplotlib
fig, axs = plt.subplots(2,3)
axs[0, 0].hist(gdf["GDP"])
axs[0, 0].set_title('GDP')
axs[0, 1].hist(gdf["social support"])
axs[0, 1].set_title("social support")
axs[1, 0].hist(gdf["healthy life"])
axs[1, 0].set_title('healthy life')
axs[1, 1].hist(gdf["freedom of choice"])
axs[1, 1].set_title('freedom of choice')
axs[0, 2].hist(gdf["generosity"])
axs[0, 2].set_title('generosity')
axs[1, 2].hist(gdf["corruption"])
axs[1, 2].set_title('corruption')
fig.tight_layout()

box plots showing outliers

In [ ]:
print("GPD boxplot")
plt.boxplot(gdf["GDP"])
plt.show()
print("social supprt boxplot")
plt.boxplot(gdf["social support"])
plt.show()
print("healthy life boxplot")
plt.boxplot(gdf["healthy life"])
plt.show()
print("freedom of choice boxplot")
plt.boxplot(gdf["freedom of choice"])
plt.show()
print("generosity boxplot")
plt.boxplot(gdf["generosity"])
plt.show()
print("corruption boxplot")
plt.boxplot(gdf["corruption"])
plt.show()

rescaling the features and preparing them for training

In [ ]:
targets = gdf['country']

values = gdf.drop(['country'], axis =1)

In [ ]:
#i will rescale all features to values in the range [0,1] (because it is easier for people to make a choice on that scale)
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
scaled_values =min_max_scaler.fit_transform(values)
scaled_values

# D3 

# trying a K-NN classifier

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.neighbors import KNeighborsClassifier
counter = 0
for i in [(1,'uniform'),(3,'distance'),(4,'uniform'),(10,'distance'),(20,'uniform'),(30,'distance'),(100,'uniform'),(150,'uniform'),(200,'uniform'),(7,'distance'),(66,'uniform'),(11,'distance'),(6,'uniform'),(43,'distance'),(10,'uniform'),(15,'uniform')]:
    counter += 1
    neighbors_num = i[0]
    weights = i[1]
    classifier = KNeighborsClassifier(n_neighbors=neighbors_num, weights=weights)
    classifier.fit(scaled_values, targets)
    prediction = classifier.predict(scaled_values)
    mean_accuracy = np.mean(prediction == targets)
    print("model number ",counter,"- mean accuracy of model with ",i[0], " neighbour(s) and",i[1]," weighting is:",  mean_accuracy)
    ####
    cv_results = cross_validate(classifier, scaled_values, targets, cv=10, 
                            return_train_score=True)

    print('Mean test score of model: {:.3f} (std: {:.3f})'
          '\nMean train score: {:.3f} (std: {:.3f})'.format(
                                                  np.mean(cv_results['test_score']),
                                                  np.std(cv_results['test_score']),
                                                  np.mean(cv_results['train_score']),
                                                  np.std(cv_results['train_score'])))
    ####
    
   
    print("box plot of cross-validation analysis:")
    plt.boxplot(cv_results['test_score'])
    plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics 

def learn_kNN_classifier(features_data, targets, neighbors, voting):
    '''Set up a K-NN classifier and fits it to the given training data.
       Return: learned classifier.''' 
    
    classifier = KNeighborsClassifier(n_neighbors=neighbors, 
                                      weights=voting)
    classifier.fit(features_data, targets)
    
    return classifier


def get_predictions(test_data, targets, classifier):
    '''Input: trained classifier, test set for feature data paired with target values.
       Return: prediction labels and the mean accuracy over the test set.'''
    
    Y_prediction = classifier.predict(test_data)
    
    accuracy = metrics.accuracy_score(targets, Y_prediction)
    
    return (prediction, accuracy)

In [ ]:
 classi = learn_kNN_classifier(scaled_values, targets, 10, 'uniform')

In [ ]:
countries =  dg['Country or region']
countries =  list(countries)

# clustering

In [ ]:
data_train, data_test, labels_train, labeles_test =  train_test_split( values, targets,train_size= 0.8, shuffle=True)
labels_train = np.array(labels_train).reshape((-1,1))

In [ ]:
from sklearn.cluster import KMeans

n_clusters = 47
kmeans = KMeans(n_clusters=n_clusters)

kmeans.fit(scaled_values)

labels = np.array(countries)

centers = kmeans.cluster_centers_

centers_labels = kmeans.predict(centers)

In [ ]:
print(labels)

In [ ]:
print(centers_labels)

In [ ]:
kmeans.predict(data_train)

# MLP 

In [ ]:
y
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = gdf['country'].values
transfomed_label = encoder.fit_transform(y)
print(y)
print(max(transfomed_label))


In [ ]:

from sklearn.model_selection import train_test_split
x_data = gdf.drop(['country'], axis =1)
x = (x_data - np.min(x_data))/(np.max(x_data)-np.min(x_data))
x_train, x_test, y_train, y_test = train_test_split(scaled_values, transfomed_label,test_size = 0.35,random_state=1)


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
clf = MLPClassifier(random_state=1, max_iter=10000).fit(scaled_values, transfomed_label)
mlp_prediction = clf.predict[]

In [ ]:
clf.score(x_test, y_test)

In [ ]:
y[clf.predict([x_test[5]])]

I will be using the MLP as the final classifier as it is highly dimensional and it also provided the highest accuracy (64%)

# random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=100,random_state=1)
rf.fit(x_train,y_train)
prediction_rf=rf.predict(x_test)

In [ ]:
rf.predict([[0.5,0.2,0.4,0.6,0.2,0.5]])

In [ ]:
x_test[2]

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns
matplotlib.rcParams['figure.dpi']= 500
rf_cm=confusion_matrix(y_test,prediction_rf)

ax = sns.heatmap(rf_cm, square=True, annot=True, fmt='d', cbar=True,
                     xticklabels=countries,
                     yticklabels=countries, cmap="seismic", annot_kws={"size":3})

plt.ylim(47, 0)

cbar = ax.collections[0].colorbar
ax.tick_params(axis='both', which='major', labelsize=5)


plt.xlabel('True label', fontsize=8)
plt.ylabel('Predicted label', fontsize=8)
plt.show()
accuracy = metrics.accuracy_score(y_test, prediction_rf)
print("the accuracy for this model is ", accuracy*100)

a 57% accuracy for such model is pretty good!

# SVM

In [ ]:
from sklearn.svm import SVC
svm=SVC(random_state=1,probability=True)
svm.fit(x_train,y_train)
svm_prediction=svm.predict(x_test)

In [ ]:
svm.predict([x_test[0]])[0]

In [ ]:
x_test[0]

In [ ]:
h = svm.predict_proba([x_test[55]])
h = np.array(h)
h = np.transpose(h)
max(h)

In [ ]:
results = svm.predict_proba(x_test)[0]

# gets a dictionary of {'class_name': probability}
prob_per_class_dictionary = dict(zip(svm.classes_, results))

# gets a list of ['most_probable_class', 'second_most_probable_class', ..., 'least_class']
results_ordered_by_probability = map(lambda x: x[0], sorted(zip(svm.classes_, results), key=lambda x: x[1], reverse=True))


In [ ]:
from sklearn import metrics
svm_cm=confusion_matrix(y_test,svm_prediction)


ax = sns.heatmap(svm_cm, square=True, annot=True, fmt='d', cbar=True,
                     xticklabels=countries,
                     yticklabels=countries, cmap="seismic", annot_kws={"size":3})

plt.ylim(47, 0)

cbar = ax.collections[0].colorbar
ax.tick_params(axis='both', which='major', labelsize=5)

plt.xlabel('True label', fontsize=8)
plt.ylabel('Predicted label', fontsize=8)
plt.show()
accuracy = metrics.accuracy_score(y_test, svm_prediction)
print("the accuracy for SVM is: ",accuracy*100)


# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors =13) # n_neighbors = k
knn.fit(x_train,y_train)
knn_prediction = knn.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
cm_knn=confusion_matrix(y_test,knn_prediction)
ax = sns.heatmap(cm_knn, square=True, annot=True, fmt='d', cbar=True,
                     xticklabels=countries,
                     yticklabels=countries, cmap="seismic", annot_kws={"size":3})

plt.ylim(47, 0)

cbar = ax.collections[0].colorbar
ax.tick_params(axis='both', which='major', labelsize=5)

plt.xlabel('True label', fontsize=8)
plt.ylabel('Predicted label', fontsize=8)
plt.show()
accuracy = metrics.accuracy_score(y_test, knn_prediction)
print("the accuracy for this model is: ",accuracy*100)


# interface

In [ ]:
import tkinter as tk
from tkinter import simpledialog

In [ ]:
class svmclassifier:
    
    #def __init__(self):
        
    
    def mainscreen(self):    
        self.window = tk.Tk()
        self.corrpt = tk.IntVar()
        self.GDP = tk.IntVar()
        self.socialsupp = tk.IntVar()
        self.foc = tk.IntVar()
        self.genros = tk.IntVar()
        self.health = tk.IntVar()
        self.window.title("country pridictor")
        
        
        corruption = simpledialog.askinteger("input", 'how corrupt do you feel you country is on a scale from 1 to 10',parent = self.window)/10
        GDP = simpledialog.askinteger("input", 'how wealthy do you think your country is on a scale from 1 to 10',parent = self.window)/10
        sociols = simpledialog.askinteger("input", "how strong is social support in you country is on a scale from 1 to 10",parent = self.window)/10
        freedomoc = simpledialog.askinteger("input", "how free is choice in your country on a scale from 1 to 10",parent = self.window)/10
        generous = simpledialog.askinteger("input", 'how generous are people from your country is on a scale from 1 to 10',parent = self.window)/10
        helth = simpledialog.askinteger("input", 'how healthy are people in your country is on a scale from 1 to 10',parent = self.window)/105
        
        prediction = clf.predict([[GDP,sociols,helth,freedomoc,generous,corruption]])      
        
        h = clf.predict_proba([[GDP,sociols,helth,freedomoc,generous,corruption]])
        h = np.array(h)
        h = np.transpose(h)
        h1 = max(h)
        
        #predictiontxt = tk.Text(self.window)
        #predictiontxt.insert(INSERT,"with {prob} confidence you are from {country}".format(prob =h, country = prediction[0] ))
        
        print("with {prob}% confidence you are from {country}".format(prob =100*h1[0], country = y[prediction[0]] ))

        
        self.window.destroy()

        self.window.mainloop()

In [ ]:
w = svmclassifier()
w.mainscreen()

# discussion and limitations

it is clear that there are many limitations to this classifier, first and most important is that it relies on the subjective opinions of people that see things within a local scale while the data is put based on a global scale, also the method of the generation of the data is a bit questionable as it does not rely directly on survies but rather on indices, it is hard for an inde alone to decide alot of varying factors for a country, espicially that the index does not come with a corresponding variance with it, missing around with the variance of the data and choosong a more well educated value would change the results of the classifier to be drastically better in real life testing